In [1]:
import json
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
import random
import joblib
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
def generate_data(n_normal=900, n_anomalous=100):
    COMMON_PORTS = [80, 443, 22, 8080]
    SUSPICIOUS_PORTS = [6666, 9999, 1337]

    def generate_normal():
        return {
            "src_port": random.choice(COMMON_PORTS),
            "dst_port": random.randint(1024, 65535),
            "packet_size": random.randint(100, 1500),
            "duration_ms": random.randint(50, 500),
            "protocol": random.choice(["TCP", "UDP"])
        }

    def generate_anomaly():
        return {
            "src_port": random.choice(SUSPICIOUS_PORTS),
            "dst_port": random.randint(1, 1023),
            "packet_size": random.choice([0, 10, 5000]),
            "duration_ms": random.randint(1000, 5000),
            "protocol": random.choice(["TCP", "UDP"])
        }

    dataset = [generate_normal() for _ in range(n_normal)] + [generate_anomaly() for _ in range(n_anomalous)]
    random.shuffle(dataset)
    return dataset

data = generate_data()
with open("dataset/training_data.json", "w") as f:
    json.dump(data, f, indent=2)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/training_data.json'

In [ ]:
with open("dataset/training_data.json") as f:
    raw_data = json.load(f)

df = pd.DataFrame(raw_data)
df.head()

In [ ]:
def preprocess_data(df):
    df_processed = pd.get_dummies(df, columns=["protocol"], drop_first=True)
    return df_processed

df_processed = preprocess_data(df)
df_processed.head()

In [ ]:
model = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)
model.fit(df_processed)

In [ ]:
joblib.dump(model, "isolation_forest_model.joblib")

In [ ]:
predictions = model.predict(df_processed)
print(np.unique(predictions, return_counts=True))

In [ ]:
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(df_processed)

# plt.figure(figsize=(10, 6))
# plt.scatter(pca_result[:, 0], pca_result[:, 1], c=predictions, cmap='coolwarm', s=10)
# plt.title("PCA Projection with Anomaly Labels")
# plt.xlabel("PCA Component 1")
# plt.ylabel("PCA Component 2")
# plt.grid(True)
# plt.show()
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Reduce data to 2D using PCA
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(processed_df)

# Add PCA result and prediction to DataFrame
viz_df = pd.DataFrame(reduced_data, columns=["PC1", "PC2"])
viz_df["Prediction"] = predictions

palette = {1: "green", -1: "red"}

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=viz_df, x="PC1", y="PC2", hue="Prediction", palette=palette)
plt.title("Anomaly Detection Visualization with PCA")
plt.grid(True)
plt.show()

In [ ]:
df["Prediction"] = predictions
anomalies = df[df["Prediction"] == -1]
anomalies.to_csv("anomalies.csv", index=False)

print(f"{len(anomalies)} anomalies saved to 'anomalies.csv'")


In [ ]:
scores = model.decision_function(processed_df)
df["Score"] = scores
df["Prediction"] = predictions

anomalies_with_score = df[df["Prediction"] == -1]
anomalies_with_score.to_csv("anomalies_with_score.csv", index=False)

print("Top 5 anomalies by confidence:")
print(anomalies_with_score.sort_values(by="Score").head())
